In [3]:
import requests
dates = []
year = 2025
for month in range(1, 5):
    gameDates = f"https://data.ncaa.com/casablanca/schedule/volleyball-men/d1/{year}/0{month}/schedule-all-conf.json"
    response = requests.get(gameDates)
    dates.extend([x for x in response.json()['gameDates'] if x['games'] > 0])
print(len(dates))

105


In [1]:
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
from tqdm import tqdm
allDataframes = []
for date in tqdm(dates):
    cdate = date['contest_date'].split("-")
    month, day, year = cdate
    response = requests.get(f"https://www.ncaa.com/scoreboard/volleyball-men/d1/{year}/{month}/{day}")
    soup = BeautifulSoup(response.text, 'html.parser')

    rows = []
    for game in soup.find_all('ul', class_='gamePod-game-teams'):
        teams = game.find_all('li')
        if len(teams) == 2:
            team_1_name = teams[0].find('span', class_='gamePod-game-team-name').text.strip()
            team_1_score = teams[0].find('span', class_='gamePod-game-team-score').text.strip()
            team_2_name = teams[1].find('span', class_='gamePod-game-team-name').text.strip()
            team_2_score = teams[1].find('span', class_='gamePod-game-team-score').text.strip()

            rows.append([team_1_name, team_1_score, team_2_name, team_2_score])

    df = pd.DataFrame(rows, columns=['Team 1', 'Team 1 Score', 'Team 2', 'Team 2 Score'])

    allDataframes.append(df)


 10%|▉         | 10/105 [00:04<00:43,  2.17it/s]


KeyboardInterrupt: 

In [9]:
pd.concat(allDataframes, ignore_index=True).to_csv(f"{year}MensNcaaGames.csv")